# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

Добавляем нужные нам библиотеки.

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from warnings import simplefilter

Загружаем датасет.

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

Получаем общую информацию о датасете. В задании сказано. что предобработка данных не понадобится, но на всякий случай проверим пропущенные значения.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Проверим на пропуски в данных

In [4]:
df.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

Проверим на явные дубликаты

In [57]:
duplicate_row = df[df.duplicated()].count()
duplicate_row

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

Посмотрим на то, как выглядят данные.

In [5]:
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


### Вывод по главе
Данные не имеют пропусков, у всех данных - правильный тип данных. Явных дубликатов не обнаружено. Дополнительно никак улучшать качество данных не нужно - они и так сказка)

## Разбейте данные на выборки

Разбиваем данные на 3 выборки: обучающую, валидацию и тестовую. И проверяем сколько строк у нас в каждой.

In [22]:
train, valid, test = np.split(df.sample(frac=1, random_state=12345), [int(.6*len(df)), int(.8*len(df))])
print('Кол-во строк в обучающей:', train.shape[0])
print('Кол-во строк в валидации:', valid.shape[0])
print('Кол-во строк в тестовой:', test.shape[0])

Кол-во строк в обучающей: 1928
Кол-во строк в валидации: 643
Кол-во строк в тестовой: 643


In [23]:
features_train = train.drop(['is_ultra'], axis=1)
target_train = train['is_ultra']
features_valid = valid.drop(['is_ultra'], axis=1)
target_valid = valid['is_ultra']
features_test = test.drop(['is_ultra'], axis=1)
target_test = test['is_ultra']

## Исследуем модель метода k-ближайших соседей, проверяем на тестовой выборке и на валидной выборке

Написал эту строчку специально, чтобы не выскакивали FutureWarning.

In [25]:
simplefilter(action='ignore', category=FutureWarning)

In [29]:
best_model = None
best_acc = 0
best_neighb = 0
for neighb in range(1, 20):
    model = KNeighborsClassifier(n_neighbors=neighb)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid) 
    acc = accuracy_score(target_valid, predictions_valid)
    if acc > best_acc:
        best_model = model
        best_neighb = neighb
        best_acc = acc
print('accuracy наилучшей модели метода k-ближайших соседей на валидационной выборке:', best_acc, "Кол-во соседей:", best_neighb)

accuracy наилучшей модели метода k-ближайших соседей на валидационной выборке: 0.7356143079315708 Кол-во соседей: 17


Теперь проверяем нашу лучшую модель на тестовой выборке

In [56]:
model = KNeighborsClassifier(n_neighbors=17)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test) 
print('accuracy наилучшей модели метода k-ближайших соседей на тестовой выборке:',accuracy_score(target_test, predictions_test))

accuracy наилучшей модели метода k-ближайших соседей на тестовой выборке: 0.7542768273716952


## Исследуем модель классификатора дерева решений, проверяем на тестовой выборке и на валидной выборке

In [31]:
best_model = None
best_acc = 0
best_depth = 0
for depth in range(1, 20):
    model = DecisionTreeClassifier(random_state=12345,max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid) 
    acc = accuracy_score(target_valid, predictions_valid)
    if acc > best_acc:
        best_model = model
        best_acc = acc
        best_depth = depth
print('accuracy наилучшей модели классификатора дерева решений на валидационной выборке:', best_acc, "Глубина:", best_depth)

accuracy наилучшей модели классификатора дерева решений на валидационной выборке: 0.7776049766718507 Глубина: 10


Теперь проверяем нашу лучшую модель на тестовой выборке

In [54]:
model = DecisionTreeClassifier(random_state=12345,max_depth=10)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test) 
print('accuracy наилучшей модели классификатора дерева решений на тестовой выборке:',accuracy_score(target_test, predictions_test))

accuracy наилучшей модели классификатора дерева решений на тестовой выборке: 0.7947122861586314


## Исследуем модель классификатора случайного леса, проверяем на тестовой выборке и на валидной выборке

In [39]:
best_model = None
best_acc = 0
best_depth = 0
best_est = 0
for est in range(10, 101, 10):
    for depth in range (1, 20):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train,target_train)
        predictions_valid = model.predict(features_valid)
        acc = accuracy_score(target_valid, predictions_valid)
        if acc > best_acc:
            best_model = model
            best_acc = acc
            best_est = est
            best_depth = depth
print('accuracy наилучшей модели классификатора случайного леса на тестовой выборке:', best_acc, "Глубина:", best_depth, 'Деревья:', best_est)

accuracy наилучшей модели классификатора случайного леса на тестовой выборке: 0.8040435458786936 Глубина: 17 Деревья: 70


Теперь проверяем нашу лучшую модель на тестовой выборке

In [55]:
model = RandomForestClassifier(random_state=12345, n_estimators=70, max_depth=17)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test) 
print('accuracy наилучшей модели классификатора случайного леса на тестовой выборке:',accuracy_score(target_test, predictions_test))

accuracy наилучшей модели классификатора случайного леса на тестовой выборке: 0.80248833592535


## Общий вывод

Для того, чтобы решить какую модель мы будем использовать в конечном итоге, мы выполнили следующие шаги:
* разделил данные на 3 выборки: обучающую, валидационную и тестовую
* выбрал 3 модели: метод k-ближайших соседей, классификатора дерева решений, классификатора случайного леса
* после обучения, валидации и тестовой выборки получил следующие результаты:
* - accuracy наилучшей модели метода k-ближайших соседей на тестовой выборке: 0.7542768273716952
* - accuracy наилучшей модели классификатора дерева решений на тестовой выборке: 0.7947122861586314
* - accuracy наилучшей модели классификатора случайного леса на тестовой выборке: 0.80248833592535
Из чего можно сделать вывод, что самой успешной моделью будет - модель классификатора случайного леса.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
